## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
# from pymystem3 import Mystem
%pip install pymystem3
from pymystem3 import Mystem
m = Mystem()


In [2]:
#Читаем файл
credit_info = pd.read_csv('credit_info.csv', index_col=0) 
#Выводим общую информацию
credit_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

В файле присутствует 12 столбцов, при этом 2 из них состоят из чисел с "плавающей точкой" (общий трудовой стаж в днях и ежемесячный доход), 5 состоят из целочисленных значений (количество детей, возраст клиента в годах, идентификатор уровня образования, идентификатор семейного положения, имелась ли задолженность по возврату кредита), 5 состоят из объектов или строк (уровень образования, семейное положение, пол, тип занятости, цель получения кредита). При этом количество значений в столбцах не равны, соответственно можно сделать вывод, что есть пропущенные значения.  

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#Выводим тип пропущенных значений в столбце total_income и days_employed
credit_info[credit_info['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [4]:
#Выводим первые 5 строк таблицы для ознакомления
credit_info.head()



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
#Находим количество пропущенных значений во всех столбцах
credit_info.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
#Выводим тип пропущенных значений в столбце total_income и days_employed
credit_info[credit_info['total_income'].isnull()].head() 


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [7]:
#Выведем значения о количестве детей
credit_info['children'].value_counts()


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [8]:
#Выведем значения о семейном положении
credit_info['family_status_id'].value_counts()


0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [9]:
#Выведем значения об образовании
credit_info['education_id'].value_counts() 

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

### Вывод

1) Выведем на экран первые 10 строк таблицы, чтобы ознакомиться с данными.
2) Посмотрим количество пропусков в столбцах. В столбце days_employed и total_income есть пропущенные значения (NaN). Предположительно, пропуски связаны с тем, что люди были еще не устроены на работу, поэтому и у них не было дохода. В доказательство этому служит равное количество пропусков в друх столбцах. 
3) Обращаем внимание, что в столбце Children есть два значения (-1 и 20), которые выделяются из общей массы. Заменим их на модуль числа -1 и на 2.

### Замена типа данных

In [10]:
#Напишем функцию для замены значений в столбце total_income на значение медианы по соответствующему типу занятости
def median_income_level(income_type): 
    median = credit_info.loc[credit_info['income_type'] == income_type, 'total_income'].median()
    return median
#Заменяем значение на медиану по "Пенсионерам"
credit_info.loc[(credit_info['total_income'].isnull()) & (credit_info['income_type'] == 'пенсионер'), 'total_income'] = median_income_level('пенсионер')
#Заменяем значение на медиану по "сотрудникам"
credit_info.loc[(credit_info['total_income'].isnull()) & (credit_info['income_type'] == 'сотрудник'), 'total_income'] = median_income_level('сотрудник') 
#Заменяем значение на медиану по "компаньонам"
credit_info.loc[(credit_info['total_income'].isnull()) & (credit_info['income_type'] == 'компаньон'), 'total_income'] = median_income_level('компаньон') 
#Заменяем значение на медиану по "госслужащим"
credit_info.loc[(credit_info['total_income'].isnull()) & (credit_info['income_type'] == 'госслужащий'), 'total_income'] = median_income_level('госслужащий') 
#Заменяем значение на медиану по "предпринимателям"
credit_info.loc[(credit_info['total_income'].isnull()) & (credit_info['income_type'] == 'предприниматель'), 'total_income'] = median_income_level('предприниматель') 
credit_info['children'] = credit_info['children'].replace({-1:1, 20:2})
#Заменим пропущенные значения в столбце на 0. 
credit_info['days_employed'] = credit_info['days_employed'].fillna(0)
#Преоразуем типы данных в целочисленные
credit_info['total_income'] = credit_info['total_income'].astype('int') 
#Преоразуем типы данных в целочисленные
credit_info['days_employed'] = credit_info['days_employed'].astype('int') 


In [11]:
#Выведем первые 10 строк таблицы после преобразований
credit_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Так как NaN замещает отсутствующее в ячейке число и принадлежит к типу float, поэтому с ним можно проводить математические операции. Заменим значения в столбце total_income на значение медианы для каждого типа занятости. Чтобы медиана была рассчитана в зависимости от конкретной группы работников. Это позволит более корректно заполнить пропущенные значения в таблице с учетом типа занятости.  В столбце days_employed заменим значения на 0, так как он не нужен для анализа модели кредитного скоринга и на результаты не повлияет. Изменим типы данных в столбцах total_income и days_employed на целочисленные значения, так как значения копеек в доходе, а также доли в днях стажа при общем анализе данных нас не будут интересовать. 

### Обработка дубликатов

In [12]:
#Посчитаем количество значений до преобразования по уровню образования
credit_info['education'].value_counts() 
# Приведем значения в столбце Education к нижнему регистру
credit_info['education'] = credit_info['education'].str.lower()


In [13]:
#Найдем сумму дубликатов до преобразования
credit_info.duplicated().sum()  

71

In [14]:
#Выведем на просмотр все значения с дубликатами
credit_info[credit_info.duplicated()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,0,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу


In [15]:
#Удалим дубликаты в таблице
credit_info = credit_info.drop_duplicates() 
credit_info.duplicated().sum()


0

In [16]:
#Посчитаем количество значений после преобразования
credit_info['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

В исходной таблице найдены 71 задублированных значения. Удалим эти строки из таблицы для дальнейшего корректного проведения анализа. Появиться они могли из-за некорректного заполнения или повторного заполнения строк для таблицы. 

### Лемматизация

In [17]:
purposes = pd.DataFrame(data=credit_info['purpose'], columns=['purpose', 'lemmas'])
purposes = purposes.drop_duplicates().reset_index(drop=True)

#Функция для приведения слова к его словарной форме(лемме)
def find_lemmas(text): 
    lemmas = m.lemmatize(text)
    return lemmas


#Добавим еще один столбец 
purposes['lemmas'] = purposes['purpose'].apply(find_lemmas) 

#Напишем функцию для отнесения цели кредита к определенной категории
def purpose_category(row): 
    lemmas = row['lemmas']
    if 'автомобиль' in lemmas:
        return 'покупка автомобиля'
    elif ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'покупка недвижимости'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'другая категория'
    
purposes['purpose_category'] = purposes.apply(purpose_category, axis=1)
credit_info = credit_info.merge(purposes, on='purpose', how='left')
credit_info['purpose_category'].value_counts()



покупка недвижимости    10811
покупка автомобиля       4306
образование              4013
свадьба                  2324
Name: purpose_category, dtype: int64

### Вывод

Так как заёмщики указывали различные цели кредита, то правильным будет сделать лемматизацию. 
Лемматизацию проводил с помощью библиотеки с функцией лемматизации на русском языке — pymystem3 по столбцу Purpose, так как нам необходимо выделить цель кредита, для улучшения дальнейшей группировки данных.
После преобразований выделены 4 основные категории цели кредита "Покупка недвижимости", "Покупка автомобиля", "Образование", "Свадьба". 

### Категоризация данных

In [18]:
education_group = credit_info.pivot_table(index='education', values='education_id').to_dict()['education_id']
education_group

{'высшее': 0,
 'начальное': 3,
 'неоконченное высшее': 2,
 'среднее': 1,
 'ученая степень': 4}

In [19]:
#Сделаем сводную таблицу по сталбцам education и education_id
education_group = credit_info.pivot_table(index='education', values='education_id').to_dict()['education_id'] 
print(education_group)

#Сделаем сводную таблицу по сталбцам family_status и family_status_id
family_group = credit_info.pivot_table(index='family_status', values='family_status_id').to_dict()['family_status_id'] 
print(family_group)

def income_level(income):
    if income <= 27000.00:
        return 'Низкий'
    if 27000 < income <= 108000.00:
        return 'Средний'
    if 108000.00 < income <= 225000.00:
        return 'Выше среднего'
    return 'Высокий'

credit_info['income_level'] = credit_info['total_income'].apply(income_level)
credit_info['income_level'].value_counts()

{'высшее': 0, 'начальное': 3, 'неоконченное высшее': 2, 'среднее': 1, 'ученая степень': 4}
{'Не женат / не замужем': 4, 'в разводе': 3, 'вдовец / вдова': 2, 'гражданский брак': 1, 'женат / замужем': 0}


Выше среднего    12269
Средний           5399
Высокий           3774
Низкий              12
Name: income_level, dtype: int64

### Вывод

Очевидно, что из наших данных можно выделить два словаря из полей образование и семейное положение. Так как именно столбцам уровень образования и семейное положения соотвествуют их же столбцы с числовым идентификатором. 
Также проведем категоризацию по уровню дохода. Создадим классификацию уровня дохода на основе данных структуры доходов населения согласно официальной статистике, где категории "Низкий" будет соответствовать доход меньше 27000/чел, категории "Средний" будет соответствовать доход от 27000/чел до 108000/чел, категории "Выше среднего" будет соответствовать доход от 108000/чел до 225000/чел, а категории "Высокий" будет соответствовать доход свыше 225000/чел. Данная классификация более точно отражает реальность.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
#Построим сводную таблицу зависимости между наличием детей и возратом кредита в срок 
children_pivot_table = credit_info.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
children_pivot_table


,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [21]:
children_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (count, debt)  6 non-null      int64  
 1   (sum, debt)    6 non-null      int64  
 2   (mean, debt)   6 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 192.0 bytes


### Вывод

При анализе зависимости между наличием детей и возвратом кредита в срок мы можем увидеть, что у заёмщиков без детей задолженность по возврату кредитов больше, чем у заёмщиков с детьми. Это объясняется тем, что заемщиков без детей численно больше. У людей с 5 детьми нет задолженностей по кредитам. При наличии 4 детей самая большая вероятность просрочки по кредитам. Также стоит отметить, что у людей с 1 ребенком вероятность также высока и составляет примерно 9%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [22]:
#Построим сводную таблицу зависимости между семейный положением и возратом кредита в срок
family_status_pivot_table = credit_info.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean']) 
family_status_pivot_table

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2810,274,0.097509
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452


In [23]:
family_status_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Не женат / не замужем to женат / замужем
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (count, debt)  5 non-null      int64  
 1   (sum, debt)    5 non-null      int64  
 2   (mean, debt)   5 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 160.0+ bytes


### Вывод

Можно сделать вывод, согласно сводной таблице между семейный положением и возратом кредита в срок, что наибольшее количество долгов присутствует у женатых/замужних заемщиков. Это объясняется тем, что эта группа самая большая количественно. Согласно долям, самая большая вероятность возникновеня долгов по кредитам у группы "Не женатых", а также группы "Гражданский брак" и составляет чуть больше 9%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#Построим сводную таблицу зависимости между уровнем дохода и возратом кредита в срок
total_income_pivot_table = credit_info.pivot_table(index=['income_level'], values='debt', aggfunc=['count', 'sum', 'mean']) 
total_income_pivot_table

,count,sum,mean
,debt,debt,debt
income_level,,,
Высокий,3774,267,0.070747
Выше среднего,12269,1043,0.085011
Низкий,12,1,0.083333
Средний,5399,430,0.079644


In [25]:
total_income_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, Высокий to Средний
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (count, debt)  4 non-null      int64  
 1   (sum, debt)    4 non-null      int64  
 2   (mean, debt)   4 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 128.0+ bytes


### Вывод

Из сводной таблицы по уровню дохода и количеством долгов мы можем сделать вывод, что у заёмщиков с низким уровнем дохода вероятность возникновения долгов по кредитам высокая, но эту статистику нельзя принимать как правило, так как количество займов слишком низкое по сравнению с остальными группами. Самая большая вероятность возникновения задолженности по кредиту у группы "Выше среднего", а также по этой группе самое большое количество должников.

- Как разные цели кредита влияют на его возврат в срок?

In [26]:
#Построим сводную таблицу зависимости между целями кредита и возратом кредита в срок
purpose_pivot_table = credit_info.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean']) 
purpose_pivot_table

,count,sum,mean
,debt,debt,debt
purpose_category,,,
образование,4013,370,0.092200
покупка автомобиля,4306,403,0.093590
покупка недвижимости,10811,782,0.072334
свадьба,2324,186,0.080034


In [27]:
purpose_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, образование to свадьба
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (count, debt)  4 non-null      int64  
 1   (sum, debt)    4 non-null      int64  
 2   (mean, debt)   4 non-null      float64
dtypes: float64(1), int64(2)
memory usage: 128.0+ bytes


### Вывод

Из сводной таблицы по целям кредита и количеству долгов по этим кредитам можно сделать вывод, что на большие по стоимости покупки в виде "Автомобиля" или "Недвижимости" приходится большее число долгов по кредитам. Это связано в первую очередь с тем, что это самые большие по численности группы. Наибольшая вероятность возниконовения долгов у групп: "Образование" и "Покупка автомобиля" - более 9%. 

### Шаг 4. Общий вывод

В проекте были использованы различные средства обработки данных, были удалены пропуски,заменены типы данных, обработаны дубликаты, проведена лемматизация, а также категоризированы данные. 
В исходных данных было 21525 значений по 10 столбцам, а также 2 столбца с пропущенными значениями.
В рамках обработки данных были заменены пропуски в столбцах total_income и days_employed. В столбце total_income были заменены пропущенные значения в рамках каждого типа занятости. А в столбце days_employed пропуски были заполнены значением '0', так как этот столбец не был задействован в процессе анализа в дальнейшем. По этим столбцам были изменены типы данных на целочисленные значения. 
В таблице присутствовал 71 дубликат, они были удалены в процессе обработки. 
Была проведена лемматизация по столбцу purpose, а также добавлен новый столбец purpose_category, где информация о целях кредита будет представлена более сгруппировано. Выделены цели: "Покупка недвижимости", "Покупка автомобиля", "Образование", "Свадьба". 
Была проведена категоризация данных по столбцу total_income и выделены группы по уровню дохода: "Низкий", "Средний", "Выше среднего" и "Высокий". 
По итогу всего проекта можно сделать выводы:
1) У людей с 5 детьми нет задолженностей по кредитам. При наличии 4 детей самая большая вероятность просрочки по кредитам. Также стоит отметить, что у людей с 1 ребенком вероятность также высока и составляет примерно 9%.
2) Согласно долям, самая большая вероятность возникновеня долгов по кредитам у группы "Не женатых", а также группы "Гражданский брак" и составляет чуть больше 9%.
3) Самая большая вероятность возникновения задолженности по кредиту у группы "Выше среднего", т.е. от 108000/чел до 225000/чел, а также по этой группе самое большое количество должников.
4) Наибольшая вероятность возниконовения долгов у групп: "Образование" и "Покупка автомобиля" - более 9%. 